In [2]:
import requests
import pandas as pd
import json
import psycopg2
from bs4 import BeautifulSoup as bs

import os
from tqdm.auto import tqdm
import time
import copy

url = "https://dictionary.yandex.net/api/v1/dicservice.json/lookup"

In [3]:
wordforms_list = []
for file in os.listdir("/Users/nigula/LL/adjust_unigr_dict/wordforms/wf_eng"):
    if "json" in file:
        with open(os.path.join("/Users/nigula/LL/adjust_unigr_dict/wordforms/wf_eng", file), "r") as f:
            data = json.load(f)
            for en_ref_word in data.keys():
                #print(data[en_ref_word])
                wordforms_list.extend(data[en_ref_word])
                #break

In [6]:
wordforms_list.index("sparse")

38188

In [4]:
wordforms_list[:10]

['youthful',
 'youthfully',
 'youthfulnesses',
 'lesions',
 'prototypic',
 'prototypical',
 'prototypes',
 'prototypal',
 'ruefully',
 'ruefulness']

In [23]:
len(wordforms_list)

130000

In [ ]:
files = [file.split(".")[0] for file in os.listdir("/Users/nigula/LL/adjust_unigr_dict/lookup/reverso_en_ru")]

In [ ]:
s = set(wordforms_list) - set(files)
len(s)

In [ ]:
files[:10]

# 1)  получаем lookup от исходного языка

In [ ]:
big_token_dict = {
 "vertolet_token" : "dict.1.1.20191029T212638Z.e3a6524ff15de8ee.53dbf57d908343531709c5f0686045121f0fc6ae",
   "bbk_token" : "dict.1.1.20190828T215834Z.0d8a3da5e08df3a0.85290ff28457230b81cb3b73720d2a0312ca14dc", 
"vasya_2019" : "dict.1.1.20191029T212638Z.e3a6524ff15de8ee.53dbf57d908343531709c5f0686045121f0fc6ae",
"vasya_2011" : "dict.1.1.20191105T083743Z.8b9cdebb9755fa57.e42afa31c184e2ced2305b5452809e17bc19c920",
"fanyi0" : "dict.1.1.20191105T095108Z.82f72da803eeedc2.ada6293a0591bed0222d3a8045bd722915f025bb",
"fanyi00" : "dict.1.1.20191105T103102Z.3e0f76aef01d786a.92fea3781473b7c432aa49ee1f3879cb95181a63",
"prvd0" : "dict.1.1.20191105T103348Z.960e0892b02b628b.5746d7d95af92a16d15941b24410707bbb7568c4",
"prvd00" : "dict.1.1.20191105T105231Z.520f147163f275cd.f329913de7b0a719762e89ef84914cbe8c3f3d25",
"prvdfin0": "dict.1.1.20191105T105704Z.a6f94da915862b61.76003fa911b16bbad05aa655d5be9ea1f78b3e1c" 
}

In [ ]:
def get_handled_words(directory_with_handled_files):
    handled_words_from_folder = []
    for word in os.listdir(directory_with_handled_files):
        #print(word.split('.')[0])
        handled_words_from_folder.append(word.split('.')[0])
    handled_words_from_folder = set(handled_words_from_folder)
    #print(check_word, "word_in", check_word in handled_words_from_folder)
    #print("akready_handled_from_directory", len(handled_words_from_folder))
    return handled_words_from_folder

a = get_handled_words("/Users/nigula/LL/adjust_unigr_dict/lookup/yandex_lookup_en_ru")

In [ ]:
def send_request_func(from_lang,to_lang, word, header_main, login, en_from_lang, en_to_lang):
    start = time.time()
    url = "https://context.reverso.net/перевод/" + from_lang + "-" + to_lang + "/" + word 
#     print("going to find word in", url)
    response = requests.get(url, headers=header_main, data = login)
    response.encoding = 'utf-8' 
    #
    save_dir = os.path.join("/Users/nigula/LL/adjust_unigr_dict/lookup/reverso" + "_" + en_from_lang + "_" + en_to_lang,word + ".xls")
    file = open(save_dir, "wb")
    file.write(response.content)
    file.close()
#     print("saved to directory", save_dir)
    soup = bs(response.text, 'html.parser') 
    time.sleep(0.01)
#     print("request_time",time.time() - start) 
    return soup

In [ ]:
def get_definitions_reverso(word, from_lang, to_lang, print_output = False):
    pos_dict = {"adv":"adverb","adj":"adjective","n":"noun",'':'',"nm":"noun","nf":"noun","v":"verb","nn":"noun",
               "adj/nm":"adjective","nfpl":"noun","conj":"conjunction","pron":"pronoun",'adj/adv':'adjective',
               "nmpl":"noun"}
    start = time.time()
    login = {'inUserName': 'n.babakov@lingualeo.com', 'inUserPass': '33vec33'}
    header_main = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    senseword_list = {"def":{"tr":[]}}
    land_dict = {"русский":"ru","французский":"fr", "английский":"en"}
    found_saved = True
    try:
        word_dir = os.path.join("/Users/nigula/LL/adjust_unigr_dict/lookup/reverso" + "_" + land_dict[from_lang] + "_" + land_dict[to_lang],word + ".xls")
        infile = open(word_dir,"r")
#         print("found_word_in", word_dir)
        contents = infile.read()
        soup = bs(contents,'html.parser')
    except:
        soup = send_request_func(from_lang,to_lang, word, header_main, login, land_dict[from_lang], land_dict[to_lang])
        found_saved = False
    #print(soup.prettify())
    first_string_passed = False
    pos = ''
    for link in soup.find_all("a", attrs={"class" : "translation"}):
        #print(link.prettify())
        try:
            #print("||",link['data-pos'])
            pos = link['data-pos'][1:-1]
        except:
            pass
        if first_string_passed == True:
            sence_word = link.text.strip()
            if len(sence_word) >0:
                if pos not in pos_dict:
                    senseword_list["def"]['tr'].append({"text":sence_word.lower(), "pos":''})
                else:
                    senseword_list["def"]['tr'].append({"text":sence_word.lower(), "pos":pos_dict[pos]})
            #print(sence_word)
        first_string_passed = True
        
    for link in soup.find_all("div", attrs={"class" : "translation"}):
        #print(link.prettify())
        try:
            #print("||",link['data-pos'])
            pos = link['data-pos'][1:-1]
        except:
            pass
        if first_string_passed == True:
            sence_word = link.text.strip()
            if len(sence_word) >0:
                if pos not in pos_dict:
                    senseword_list["def"]['tr'].append({"text":sence_word.lower(), "pos":''})
                else:
                    senseword_list["def"]['tr'].append({"text":sence_word.lower(), "pos":pos_dict[pos]})
            #print(sence_word)
        first_string_passed = True
    if len (senseword_list) == 0:
        print("turn to alternative marks")
        for link in soup.find_all("div", attrs={"lang" : "fr"}):
            try:
                #print("||",link['data-pos'])
                pos = link['data-pos'][1:-1]
            except:
                pass
            sence_word = link.text.strip()
            senseword_list["def"]['tr'].append({"text":sence_word.lower(), "pos":pos})
            #print(link.prettify())
#     print("definiotns_time", time.time() - start)
    return senseword_list, found_saved


get_definitions_reverso("Extra", "английский", "русский", print_output = False)



In [ ]:
def get_y_lookup_json(from_lang_words_list, from_lang, to_lang, token_dict, directory_with_handled_files):
    tokens_names_list = []
    tokens_list = []
    for token_name, token in token_dict.items():
        tokens_names_list.append(token_name)
        tokens_list.append(token)
    token_index = 0 
    not_handled_words_list = []
    
    #save_folder = "/Users/nigula/LL/adjust_unigr_dict/lookup/yandex_lookup_fr_en_2"
    save_folder = directory_with_handled_files
   
    handled_words_from_folder = get_handled_words(directory_with_handled_files)
    handled_words_from_folder = set(handled_words_from_folder)
    handled_words_from_folder_2 = get_handled_words(save_folder)
    handled_words_from_folder_2 = set(handled_words_from_folder_2)
    handled_words_from_folder = handled_words_from_folder.union(handled_words_from_folder_2)
    print("handled_all_words", len(handled_words_from_folder))

    count = 0 
    print("going_to_use token", tokens_names_list[token_index])
    skipped_words_count = 0
    handled_words_count = 0
    offline_handled_reverso = 0
    must_request_reverso = 0
    for fr_word in tqdm(from_lang_words_list):
        #print(fr_word)
        isnan = False
        #print(count, start_from_count)
        try:
            isnan = math.isnan(fr_word)
        except:
            pass
        #print(fr_word != "no_equality", isnan == False , fr_word not in handled_words_from_folder)
        if fr_word != "no_equality" and isnan == False and fr_word not in handled_words_from_folder:
            lang_pair = from_lang + "-" + to_lang
            headers = {"key":tokens_list[token_index],
                  "lang":lang_pair,"text":fr_word}
            ddd = requests.get(url, headers).json()
            if 'def' in ddd:
                if len(ddd['def']) == 0 or len (ddd['def']['tr']==0):
                    #print("go_to_reverso")
                    land_dict = {"ru":"русский","fr":"французский", "en":"английский"}
                    ddd, found_in_folder = get_definitions_reverso(fr_word,land_dict[from_lang],  land_dict[to_lang])
            else:
                land_dict = {"ru":"русский","fr":"французский", "en":"английский"}
                ddd, found_in_folder = get_definitions_reverso(fr_word,land_dict[from_lang],  land_dict[to_lang])
            if 'message' in ddd and ddd['message'] == "Limit of daily requests exceeded":
                token_index += 1
                print("LIMIT REACHED, switch to token",tokens_names_list[token_index] )
                not_handled_words_list.append(fr_word)
            try:
                save_loc = os.path.join(save_folder, fr_word + ".json")
            except:
                print(fr_word, "is ambigious float or smth")
                continue
            try:
                with open(save_loc, 'w') as f:
                    json.dump(ddd, f, indent = 4, ensure_ascii=False)
                    #print("saved at ", save_loc)
            except Exception as E:
                print(fr_word, E)
            #time.sleep(0.0001)
            handled_words_count += 1
            if found_in_folder == True:
                offline_handled_reverso += 1
            else:
                must_request_reverso += 1
        else:
            skipped_words_count += 1
            #print(fr_word, "===>>>>skipped")
            pass
            
        count += 1
        if count %5000 == 0:
            handled_words_from_folder = get_handled_words(directory_with_handled_files)
            handled_words_from_folder = set(handled_words_from_folder)
            handled_words_from_folder_2 = get_handled_words(save_folder)
            handled_words_from_folder_2 = set(handled_words_from_folder_2)
            handled_words_from_folder = handled_words_from_folder.union(handled_words_from_folder_2)
            print("offline_handled_reverso", offline_handled_reverso,"must_request_reverso",must_request_reverso )
            print("handled_words_from_folder", len(handled_words_from_folder))
            print(handled_words_count, "words handled", skipped_words_count, "words skipped\n",)     
    return not_handled_words_list
        

In [ ]:
with op

In [ ]:
wordforms_list[:10]

In [ ]:
handled = "/Users/nigula/LL/adjust_unigr_dict/lookup/yandex_lookup_en_ru_wordforms/"
#from_ind = 500 + 689 + 229 + 3467 + 425 + 11717 + 72554
from_ind = 0
to_ind = len(wordforms_list)
not_handled_words = get_y_lookup_json(wordforms_list[from_ind:to_ind], "en", "ru", big_token_dict,
                  directory_with_handled_files = handled)

In [ ]:
 len(os.listdir("/Users/nigula/LL/adjust_unigr_dict/lookup/yandex_lookup_en_ru_wordforms/"))

In [ ]:
handled = "/Users/nigula/LL/adjust_unigr_dict/lookup/yandex_lookup_fr_en"
#from_ind = 1480+382+133+1858+15
#to_ind = 25000
from_ind = 0
to_ind = len(words_to_be_handled)
not_handled_words = get_y_lookup_json(words_to_be_handled[from_ind:to_ind], "fr", "en", small_token_dict,
                  directory_with_handled_files = han dled)



# 2) объединяем сохраненные файлы в csv


In [75]:
for i in range(10):
    if i == 4: 
        print("skip_this_iter")
        continue
        
    print(i)
    print("next_iter")

0
next_iter
1
next_iter
2
next_iter
3
next_iter
skip_this_iter
5
next_iter
6
next_iter
7
next_iter
8
next_iter
9
next_iter


In [76]:
def parse_from_yandex_lookup(directory, print_output = False, necessary_words_list = None):
    word = []
    local_word = []
    pos_list = []
    examples = []
    examples_local = []
    words_from_not_handled_count = 0 
    empty = 0
    skipped_words = 0
    for file in tqdm(os.listdir(directory)):
        if print_output == True: print(file)
        current_word = file.split(".")[0]
        
        if "DS_Store" not in file:
            if necessary_words_list != None:
                curr_handled_word = file.split(".")[0]
                if curr_handled_word not in necessary_words_list:
                    skipped_words += 1
                    continue
            try:
                f = open(os.path.join(directory,file), "r", encoding='utf-8')
            except:
                f = open(os.path.join(directory,file), "r")
            try:
                data = json.load(f, encoding = 'utf-8')
            except:
                data = json.load(f)
            if print_output == True:print(data)
            if "head" in data:
                if print_output == True:print("YANDEX")
                for definition in data['def']:
#                     print(definition)
                    word_current =  definition['text'] 
                    if 'pos' in definition:
                        global_pos = definition['pos']
                    else:
                        global_pos = 'no_pos_available'
                    for translation in definition['tr']: 
                        if 'pos' in translation:
                            pos = translation['pos']
                        else:
                            pos = global_pos
                        if print_output == True:print(pos, word_current, translation['text'])
                        pos_list.append(pos)
                        word.append(word_current)
                        local_word.append(translation['text'])
                        ex_en = []
                        ex_rus = []
#                         if 'ex' in translation:
#                             for exmpl in translation['ex']:
#                                 #print(exmpl)
#                                 ex_en.append(exmpl['text'])
#                                 ex_rus.append(exmpl['tr'][0]['text'])
#                         examples.append(ex_en)
#                         examples_local.append(ex_rus)
                if len(data['def']) == 0:
                    empty += 1
            else:
                if print_output == True:print("CONTEXT_REVERSO")
                word_current = file.split(".")[0]
                for definition in data['def'].values():
                    for translation in definition:
                        if translation['pos'] == '':
                            pos = 'no_pos_available'
                        else:
                            pos = translation['pos']
                        if print_output == True:print(pos, word_current, translation['text'])
                        pos_list.append(pos)
                        word.append(word_current)
                        local_word.append(translation['text'])
                        
            words_from_not_handled_count += 1
    print("empty json", empty)
    print("non_handled_basic_language__words_added_to_dataframe", words_from_not_handled_count)
    print(skipped_words,"skipped_words")
    data = pd.DataFrame(list(zip(word,local_word,pos_list)),columns =['word', 'local_word', 'pos'])
    return  data


In [35]:
a = [1,2,3,3]
typle(a)

{1, 2, 3}

In [36]:
handled_eng_wods = set(df['word'])

In [37]:
len(set(df['word'])), len(os.listdir("/Users/nigula/LL/adjust_unigr_dict/lookup/yandex_lookup_en_ru_wordforms/"))

(42781, 63657)

In [77]:
df = parse_from_yandex_lookup("/Users/nigula/LL/adjust_unigr_dict/lookup/yandex_lookup_en_ru_wordforms/",
                             print_output = False, necessary_words_list = wordforms_list)


empty json 0
non_handled_basic_language__words_added_to_dataframe 63654
2 skipped_words


In [78]:
df.head()

,word,local_word,pos
0,longs,жаждет,no_pos_available
1,longs,тоскует,no_pos_available
2,longs,мечтает,no_pos_available
3,longs,страстно желает,no_pos_available
4,instilments,частями,no_pos_available


In [79]:
set(df['word'])

{'thermodynamicss',
 'fording',
 'painfully',
 'timorously',
 'accordance',
 'alternative',
 'nastiness',
 'condolent',
 'emotions',
 'melody',
 'bestows',
 'balconies',
 'nests',
 'gnashing',
 'quarantines',
 'sighted',
 'mysticism',
 'mittens',
 'cranes',
 'repetitiveness',
 'lengthiness',
 'pom-poms',
 'grandness',
 'namelessness',
 'throw',
 'chicnesses',
 'Windowss',
 'cuticula',
 'overlay',
 'surcharges',
 'outfighting',
 'explode',
 'hyperbolic',
 'artless',
 'literally',
 'tender',
 'banker',
 'dibss',
 'hooter',
 'antagonist',
 'ellipticity',
 'submerse',
 'exonerate',
 'progress',
 'mineralized',
 'ratified',
 'dramatized',
 'shone',
 'cashed',
 'splicer',
 'auricles',
 'vibrates',
 'antiquity',
 'circumferential',
 'conscripted',
 'hierarchies',
 'criminally',
 'espousal',
 'dissected',
 'harpist',
 'regent',
 'foaming',
 'hatter',
 'crunches',
 'disposes',
 'smartness',
 'royalty',
 'glycerines',
 'parolee',
 'venoms',
 'domiciled',
 'witnesses',
 'fictions',
 'fellowships'

In [80]:
df.to_csv("/Users/nigula/LL/adjust_unigr_dict/lookup results/yandex_lookup_en_ru_wordforms.csv")

# Sanity check

In [29]:
def check_empty_lookup(directory,handled_words, print_output = False):
    total_words_count = 0 
    empty = 0
    not_handled_from_original_list = 0
    for file in tqdm(os.listdir(directory)):
        if print_output == True: print(file)
        current_word = file.split(".")[0]
        if "DS_Store" not in file:
            try:
                f = open(os.path.join(directory,file), "r", encoding='utf-8')
            except:
                f = open(os.path.join(directory,file), "r")
            try:
                data = json.load(f, encoding = 'utf-8')
            except:
                data = json.load(f)
            if print_output == True:print(data)
            word_current = file.split(".")[0]
            if word_current not in handled_words:
                not_handled_from_original_list += 1
            if "head" in data:
                if print_output == True:print("YANDEX")
#                 for definition in data['def']:
#                     word_current =  definition['text'] 
#                     if 'pos' in definition:
#                         global_pos = definition['pos']
#                     else:
#                         global_pos = 'no_pos_available'
#                     if len(definition['tr']): 
                        
#                     for translation in definition['tr']: 
#                         if 'pos' in translation:
#                             pos = translation['pos']
#                         else:
#                             pos = global_pos
#                         if print_output == True:print(pos, word_current, translation['text'])
#                         pos_list.append(pos)
#                         word.append(word_current)
#                         local_word.append(translation['text'])
#                         ex_en = []
#                         ex_rus = []
                if len(data['def']) == 0:
                    empty += 1
            else:
                if print_output == True:print("CONTEXT_REVERSO")
                
                if len(data['def']['tr']) == 0:
                    empty += 1
#                 for definition in data['def'].values():
#                     for translation in definition:
#                         if translation['pos'] == '':
#                             pos = 'no_pos_available'
#                         else:
#                             pos = translation['pos']
#                         if print_output == True:print(pos, word_current, translation['text'])
#                         pos_list.append(pos)
#                         word.append(word_current)
#                         local_word.append(translation['text'])       
            total_words_count += 1
    print(empty, "empty words",total_words_count,"total_words_count", not_handled_from_original_list,"not_handled_from_original_list" )


In [42]:
len(wordforms_list),len(set(wordforms_list))

(130000, 64005)

In [38]:
check_empty_lookup("/Users/nigula/LL/adjust_unigr_dict/lookup/yandex_lookup_en_ru_wordforms/", handled_eng_wods)

20880 empty words 63656 total_words_count 21008 not_handled_from_original_list


# * делаем список ссылок на запрос, сохраняем асинхронными парсингом

In [46]:
df.head()

,word,local_word,pos
0,longs,жаждет,no_pos_available
1,longs,тоскует,no_pos_available
2,longs,мечтает,no_pos_available
3,longs,страстно желает,no_pos_available
4,instilments,частями,no_pos_available


In [47]:
def get_link(from_lang,to_lang, word):
    url = "https://context.reverso.net/перевод/" + from_lang + "-" + to_lang + "/" + word 
    return url

In [51]:


wordforms_list = []
for file in os.listdir("/Users/nigula/LL/adjust_unigr_dict/wordforms/wf_eng"):
    if "json" in file:
        with open(os.path.join("/Users/nigula/LL/adjust_unigr_dict/wordforms/wf_eng", file), "r") as f:
            data = json.load(f)
            for en_ref_word in data.keys():
                #print(data[en_ref_word])
                wordforms_list.extend(data[en_ref_word])
                #break




In [56]:
rus_wordforms = list(set(df['local_word']))

In [57]:
def get_non_handled_words_link(handled_directory, words_list, from_lang, to_lang):
    handled_word_list = []
    for handled_word in os.listdir(handled_directory):
        word = handled_word.split(".")[0]
        handled_word_list.append(word)

    links_list = []
    for word in tqdm(list(set(words_list))):
        if word not in handled_word_list:
            url = get_link(from_lang, to_lang, word)
            links_list.append(url)
    return links_list

links_list = get_non_handled_words_link("/Users/nigula/LL/adjust_unigr_dict/lookup/reverso_ru_fr", rus_wordforms,"русский", "французский")


In [58]:
links_list[:10]

['https://context.reverso.net/перевод/русский-французский/заболачивание',
 'https://context.reverso.net/перевод/русский-французский/составил почти',
 'https://context.reverso.net/перевод/русский-французский/отрубленной',
 'https://context.reverso.net/перевод/русский-французский/затоптали',
 'https://context.reverso.net/перевод/русский-французский/рубежи',
 'https://context.reverso.net/перевод/русский-французский/предупреждающее',
 'https://context.reverso.net/перевод/русский-французский/лет двадцати',
 'https://context.reverso.net/перевод/русский-французский/кэмпбеллов',
 'https://context.reverso.net/перевод/русский-французский/latsch',
 'https://context.reverso.net/перевод/русский-французский/охватывать']

In [59]:
with open("reverso_ru_fr_links_list.json", "w") as f:
    data = json.dump(links_list,f)

In [62]:
with open("reverso_en_fr_links_list.json", "r") as f:
    data = json.load(f)
len(data)

64005

# * Запускаем скрипт cross_approach_script к полученному csv. Меняем параметры

# * объединяем с прошлым 1)concat_medium_results_and_detect_missing_stuff 2) merge_cross_approach

In [ ]:
лемматизируем всю херню и добавляем в отдельный столбец для каждого языка
